In [3]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local', 'pyspark')
import numpy

In [15]:
from Similarities import euc_dist

rdd = sc.textFile("train_no_header.csv")
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[0],[x[1],int(x[2])]))

rdd = rdd.groupByKey()
rdd = rdd.map(lambda x:(x[0],list(x[1])))
sim_rdd=rdd.cartesian(rdd)
sim_rdd=sim_rdd.filter(lambda x: x[0][0] > x[1][0])
sim_rdd=sim_rdd.map(lambda x: (x[0][0],x[1][0],euc_dist(x[0][1],x[1][1]))).filter(lambda x:x[2]>0)
sim_rdd.take(10)

[(u'13357', u'10539', 0.5),
 (u'13357', u'10121', 0.5),
 (u'13357', u'11513', 0.5),
 (u'13357', u'13005', 0.5),
 (u'13357', u'12395', 0.5),
 (u'13357', u'11961', 0.5),
 (u'13357', u'1236', 0.5),
 (u'13357', u'13010', 0.5),
 (u'4447', u'13612', 0.5),
 (u'11542', u'10570', 1.0)]

In [20]:
urm_rdd=(sc
         .textFile("train_no_header.csv")
         .map(lambda x: x.split(','))
         .map(lambda x:(x[0],[x[1],int(x[2])])))

sim_rdd=sim_rdd.map(lambda x: (x[0],[x[1],x[2]]))
              
sim_rdd.take(10)

[(u'13357', [u'10539', 0.5]),
 (u'13357', [u'10121', 0.5]),
 (u'13357', [u'11513', 0.5]),
 (u'13357', [u'13005', 0.5]),
 (u'13357', [u'12395', 0.5]),
 (u'13357', [u'11961', 0.5]),
 (u'13357', [u'1236', 0.5]),
 (u'13357', [u'13010', 0.5]),
 (u'4447', [u'13612', 0.5]),
 (u'11542', [u'10570', 1.0])]

In [21]:
urm_rdd.take(10)

[(u'2738', [u'1', 1]),
 (u'4716', [u'1', 4]),
 (u'13298', [u'1', 8]),
 (u'15122', [u'1', 4]),
 (u'11326', [u'2', 5]),
 (u'3406', [u'3', 4]),
 (u'3906', [u'3', 10]),
 (u'6345', [u'3', 6]),
 (u'7393', [u'3', 8]),
 (u'815', [u'4', 8])]

In [22]:
join_rdd=sim_rdd.join(urm_rdd)
join_rdd.take(10)

[(u'4446', ([u'12010', 1.0], [u'14644', 10])),
 (u'4446', ([u'12010', 1.0], [u'16547', 10])),
 (u'4446', ([u'12010', 1.0], [u'22546', 10])),
 (u'4446', ([u'12835', 1.0], [u'14644', 10])),
 (u'4446', ([u'12835', 1.0], [u'16547', 10])),
 (u'4446', ([u'12835', 1.0], [u'22546', 10])),
 (u'4446', ([u'12877', 1.0], [u'14644', 10])),
 (u'4446', ([u'12877', 1.0], [u'16547', 10])),
 (u'4446', ([u'12877', 1.0], [u'22546', 10])),
 (u'4446', ([u'14182', 1.0], [u'14644', 10]))]

In [29]:
def compute_rec (x):
    num=sum(i[0]*i[1] for i in x[1])
    den=sum(i[0] for i in x[1])
    if den==0:
        return 0
    return x[0],float(num)/den

rec_rdd=(join_rdd
         .map(lambda x: ((x[1][0][0],x[1][1][0]),(x[1][0][1],x[1][1][1])))
         .groupByKey())
         #.map(compute_rec))
rec_rdd.take(10)


[((u'5775', u'14169'), <pyspark.resultiterable.ResultIterable at 0x106152210>),
 ((u'11677', u'30533'),
  <pyspark.resultiterable.ResultIterable at 0x106152d50>),
 ((u'11283', u'29073'),
  <pyspark.resultiterable.ResultIterable at 0x106152690>),
 ((u'12581', u'13379'),
  <pyspark.resultiterable.ResultIterable at 0x1061523d0>),
 ((u'10880', u'15575'),
  <pyspark.resultiterable.ResultIterable at 0x106088c90>),
 ((u'12159', u'7974'), <pyspark.resultiterable.ResultIterable at 0x106088890>),
 ((u'2843', u'16481'), <pyspark.resultiterable.ResultIterable at 0x106088b90>),
 ((u'13184', u'643'), <pyspark.resultiterable.ResultIterable at 0x10601cbd0>),
 ((u'1236', u'17539'), <pyspark.resultiterable.ResultIterable at 0x106106bd0>),
 ((u'10106', u'34058'),
  <pyspark.resultiterable.ResultIterable at 0x106106d50>)]

In [30]:
rec_rdd=rec_rdd.map(compute_rec)
rec_rdd.take(10)

[((u'2913', u'18168'), 7.8),
 ((u'3891', u'12290'), 8.0),
 ((u'11641', u'25084'), 7.666666666666667),
 ((u'14362', u'23520'), 3.0),
 ((u'5824', u'33741'), 8.0),
 ((u'6714', u'30806'), 8.0),
 ((u'11051', u'17747'), 7.060660171779822),
 ((u'10345', u'30729'), 5.0),
 ((u'2089', u'26307'), 10.0),
 ((u'12475', u'31816'), 4.0)]